# Save/Load ForecastFlowML

This guide shows how the ``ForecastFlowML`` can be saved and loaded to be used afterwards.

## Import packages

In [1]:
from forecastflowml import ForecastFlowML
from forecastflowml import FeatureExtractor
from forecastflowml.data.loader import load_walmart_m5
from lightgbm import LGBMRegressor
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pickle

## Initialize Spark

In [2]:
spark = (
    SparkSession.builder.master("local[4]")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.sql.execution.pyarrow.enabled", "true")
    .getOrCreate()
)

## Sample Dataset

In [3]:
df = load_walmart_m5(spark)
df.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|
+--------------------+-----------+-------+------+--------+--------+----------+-----+
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-01|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-02|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-03|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-04|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-05|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-06|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-07|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-08|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      T

## Feature Engineering

In [4]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    lag_window_features={
        "lag": [7 * (i + 1) for i in range(4)],
    },
)
df_features = feature_extractor.transform(df).localCheckpoint()
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+-----+------+------+------+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|lag_7|lag_14|lag_21|lag_28|
+--------------------+-----------+-------+------+--------+--------+----------+-----+-----+------+------+------+
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-01|  0.0| null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-02|  0.0| null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-03|  0.0| null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-04|  0.0| null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-05|  0.0| null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-06|  0.0| null|  null|  null|

## Train/Test Dataset

In [5]:
df_train = df_features.filter(F.col("date") < "2016-04-25")
df_test = df_features.filter(F.col("date") >= "2016-04-25")

## Initialize Model

In [6]:
forecast_flow = ForecastFlowML(
    group_col="store_id",
    id_col="id",
    date_col="date",
    target_col="sales",
    date_frequency="days",
    model_horizon=7,
    max_forecast_horizon=28,
    model=LGBMRegressor(),
)

### PySpark DataFrame with Distributed Results

#### Save

In [7]:
forecast_flow.train(df_train).write.parquet("trained_models.parquet")
with open("forecast_flow.pickle", "wb") as f:
    pickle.dump(forecast_flow, f)

#### Load

In [8]:
trained_models = spark.read.parquet("trained_models.parquet")
with open("forecast_flow.pickle", "rb") as f:
    forecast_flow = pickle.load(f)
forecast_flow.predict(df_test, trained_models).show(10)

+-----+--------------------+----------+----------+
|group|                  id|      date|prediction|
+-----+--------------------+----------+----------+
| CA_1|FOODS_1_064_CA_1_...|2016-04-25|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-26|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-27|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-28| 1.0461295|
| CA_1|FOODS_1_064_CA_1_...|2016-04-29| 1.6917489|
| CA_1|FOODS_1_064_CA_1_...|2016-04-30| 2.5920935|
| CA_1|FOODS_1_064_CA_1_...|2016-05-01| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-25| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-26| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-27| 1.0461295|
+-----+--------------------+----------+----------+
only showing top 10 rows



### PySpark DataFrame with Local Results

#### Save

In [9]:
forecast_flow.train(df_train, local_result=True)
with open("forecast_flow.pickle", "wb") as f:
    pickle.dump(forecast_flow, f)

#### Load

In [10]:
with open("forecast_flow.pickle", "rb") as f:
    forecast_flow = pickle.load(f)
forecast_flow.predict(df_test, spark=spark).show(10)

+-----+--------------------+----------+----------+
|group|                  id|      date|prediction|
+-----+--------------------+----------+----------+
| CA_1|FOODS_1_064_CA_1_...|2016-04-25|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-26|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-27|0.55427927|
| CA_1|FOODS_1_064_CA_1_...|2016-04-28| 1.0461295|
| CA_1|FOODS_1_064_CA_1_...|2016-04-29| 1.6917489|
| CA_1|FOODS_1_064_CA_1_...|2016-04-30| 2.5920935|
| CA_1|FOODS_1_064_CA_1_...|2016-05-01| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-25| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-26| 1.0461295|
| CA_1|FOODS_1_121_CA_1_...|2016-04-27| 1.0461295|
+-----+--------------------+----------+----------+
only showing top 10 rows



### Pandas DataFrame

#### Save

In [11]:
forecast_flow.train(df_train.toPandas(), spark=spark)
with open("forecast_flow.pickle", "wb") as f:
    pickle.dump(forecast_flow, f)

#### Load

In [12]:
with open("forecast_flow.pickle", "rb") as f:
    forecast_flow = pickle.load(f)
forecast_flow.predict(df_test.toPandas(), spark=spark)

,group,id,date,prediction
0,CA_1,FOODS_1_064_CA_1_evaluation,2016-04-25,0.554279
1,CA_1,FOODS_1_064_CA_1_evaluation,2016-04-26,0.554279
2,CA_1,FOODS_1_064_CA_1_evaluation,2016-04-27,0.554279
3,CA_1,FOODS_1_064_CA_1_evaluation,2016-04-28,1.046129
4,CA_1,FOODS_1_064_CA_1_evaluation,2016-04-29,1.691749
...,...,...,...,...
5707,WI_1,HOUSEHOLD_2_510_WI_1_evaluation,2016-05-18,0.522341
5708,WI_1,HOUSEHOLD_2_510_WI_1_evaluation,2016-05-19,1.890244
5709,WI_1,HOUSEHOLD_2_510_WI_1_evaluation,2016-05-20,0.522341
5710,WI_1,HOUSEHOLD_2_510_WI_1_evaluation,2016-05-21,0.522341
